# Trabajo Práctico Final

In [4]:
import numpy as np
from numpy.linalg import det, inv
from sklearn.datasets import load_iris, fetch_openml
from sklearn.model_selection import train_test_split

In [5]:
class ClassEncoder:
    def fit(self, y):
        self.names = np.unique(y)
        self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
        self.fmt = y.dtype

    def _map_reshape(self, f, arr):
        return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)

    def transform(self, y):
        return self._map_reshape(lambda name: self.name_to_class[name], y)

    def fit_transform(self, y):
        self.fit(y)
        return self.transform(y)

    def detransform(self, y_hat):
        return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [6]:
class BaseBayesianClassifier:
    def __init__(self):
        self.encoder = ClassEncoder()

    def _estimate_a_priori(self, y):
        a_priori = np.bincount(y.flatten().astype(int)) / y.size
        return np.log(a_priori)

    def _fit_params(self, X, y):
        raise NotImplementedError()

    def _predict_log_conditional(self, x, class_idx):
        raise NotImplementedError()

    def fit(self, X, y, a_priori=None):
        y = self.encoder.fit_transform(y)
        self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)
        assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"
        self._fit_params(X, y)

    def predict(self, X):
        m_obs = X.shape[1]
        y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
        for i in range(m_obs):
            encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
            y_hat[i] = self.encoder.names[encoded_y_hat_i]
        return y_hat.reshape(1,-1)

    def _predict_one(self, x):
        log_posteriori = [log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                          in enumerate(self.log_a_priori)]
        return np.argmax(log_posteriori)

In [7]:
class QDA(BaseBayesianClassifier):
    def _fit_params(self, X, y):
        self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                         for idx in range(len(self.log_a_priori))]
        self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]

    def _predict_log_conditional(self, x, class_idx):
        inv_cov = self.inv_covs[class_idx]
        unbiased_x =  x - self.means[class_idx]
        return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [8]:
class TensorizedQDA(QDA):
    def _fit_params(self, X, y):
        super()._fit_params(X,y)
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x
        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

## Funciones auxiliares

In [9]:
def get_iris_dataset():
    data = load_iris()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_penguins():
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')
    df.drop(columns=["island","sex"], inplace=True)
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]
    return df.values, tgt.to_numpy().reshape(-1,1)

def split_transpose(X, y, test_sz, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sz, random_state=random_state)
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

## Ejemplo de entrenamiento

In [10]:
# hiperparámetros
rng_seed = 6543

# Cargar y preparar los datos
X_full, y_full = get_iris_dataset()
print(f"X: {X_full.shape}, Y:{y_full.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, rng_seed)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

# Entrenar un modelo QDA
qda = QDA()
qda.fit(train_x, train_y)

# Evaluar el modelo
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

X: (150, 4), Y:(150, 1)
(4, 90) (1, 90) (4, 60) (1, 60)
Train (apparent) error is 0.0111 while test error is 0.0167


## Ejemplos de medición de tiempos

In [ ]:
%%timeit

qda.predict(test_x)

In [ ]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

## Implementación base
1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
2. Repetir el punto anterior para el dataset *penguin*.
3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA (no múltiples prioris) ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?
5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?


**Sugerencia:** puede resultar de utilidad para cada inciso de comparación utilizar tablas del siguiente estilo:

<center>

Modelo | Dataset | Seed | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | 0.55 | 0.85
LDA | Iris | 125 | 0.22 | 0.8

</center>


### 1. Entrenar QDA sobre el dataset iris con diferentes distribuciones a priori

In [12]:
# Se instancia QDA
qda = QDA()

# Se genera un array con las probabilidades de las distintas clases para el dataset iris
probability_arrays = np.array([np.full(3,1/3),np.full(3,[.05,.05,.9]),np.full(3,[.05,.9,.05]),np.full(3,[.9,.05,.05])])
for prob_array in probability_arrays:

    qda.fit(train_x, train_y,prob_array)
    train_acc = accuracy(train_y, qda.predict(train_x))
    test_acc = accuracy(test_y, qda.predict(test_x))
    print(f"Train error for iris dataset and apriori probability distribution {prob_array} is {1 - train_acc:.4f}")
    print(f"Test error for the same dataset:  {1 - test_acc:.4f}")


Train error for iris dataset and apriori probability distribution [0.33333333 0.33333333 0.33333333] is 0.0222
Test error for the same dataset:  0.0167
Train error for iris dataset and apriori probability distribution [0.05 0.05 0.9 ] is 0.0333
Test error for the same dataset:  0.0500
Train error for iris dataset and apriori probability distribution [0.05 0.9  0.05] is 0.0333
Test error for the same dataset:  0.0000
Train error for iris dataset and apriori probability distribution [0.9  0.05 0.05] is 0.0222
Test error for the same dataset:  0.0167


### 2. Repetir el punto anterior para el dataset penguin

In [14]:
# Código para el punto 2

# Cargar y preparar los datos
X_penguins_full, y_penguins_full = get_penguins()
print(f"X: {X_penguins_full.shape}, Y:{y_penguins_full.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
train_x_penguins, train_y_penguins, test_x_penguins, test_y_penguins = split_transpose(X_penguins_full, y_penguins_full, 0.4, rng_seed)
print(train_x_penguins.shape, train_y_penguins.shape, test_x_penguins.shape, test_y_penguins.shape)

# Entrenar un modelo QDA
qda_penguins = QDA()
for prob_array in probability_arrays:

    qda_penguins.fit(train_x_penguins, train_y_penguins,prob_array)
    penguins_train_acc = accuracy(train_y_penguins, qda_penguins.predict(train_x_penguins))
    penguins_test_acc = accuracy(test_y_penguins, qda_penguins.predict(test_x_penguins))
    print(f"Train error for penguin dataset and apriori probability distribution {prob_array} is {1 - penguins_train_acc:.4f}")
    print(f"Test error for the same dataset:  {1 - penguins_test_acc:.4f}")


X: (342, 4), Y:(342, 1)
(4, 205) (1, 205) (4, 137) (1, 137)
Train error for penguin dataset and apriori probability distribution [0.33333333 0.33333333 0.33333333] is 0.0098
Test error for the same dataset:  0.0073
Train error for penguin dataset and apriori probability distribution [0.05 0.05 0.9 ] is 0.0098
Test error for the same dataset:  0.0073
Train error for penguin dataset and apriori probability distribution [0.05 0.9  0.05] is 0.0098
Test error for the same dataset:  0.0219
Train error for penguin dataset and apriori probability distribution [0.9  0.05 0.05] is 0.0195
Test error for the same dataset:  0.0219


### 3. Implementar LDA y comparar con QDA

In [23]:
# Código para el punto 3
class LDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    cov_matrices = [np.cov(X[:,y.flatten()==idx], bias=True) for idx in range(len(self.log_a_priori))]
    class_frequencies = np.bincount(y.flatten().astype(int)) / y.size

    weighted_cov_mean = sum(cov_matrix * weight for cov_matrix, weight in zip(cov_matrices, class_frequencies))
    self.inv_cov = inv(weighted_cov_mean)

    self.means = np.array([X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))])

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_cov
    half_unbiased_x =  x - 0.5*self.means[class_idx]
    return self.means[class_idx].T @ inv_cov @ half_unbiased_x
  
#Prueba con Iris
lda_iris = LDA()

prob_array_iris = np.full(3,1/3)

lda_iris.fit(train_x, train_y, prob_array)
train_acc = accuracy(train_y, lda_iris.predict(train_x))
test_acc = accuracy(test_y, lda_iris.predict(test_x))
print(f"Testing LDA for iris dataset and apriori probability distribution")
print(f"Train error for iris dataset and apriori probability distribution {prob_array_iris} is {1 - train_acc:.4f}")
print(f"Test error for the same dataset:  {1 - test_acc:.4f}")

#Prueba con dataset penguin
lda_penguin = LDA()

prob_array_penguin = np.full(3,1/3)

lda_penguin.fit(train_x_penguins, train_y_penguins, prob_array)
penguins_train_acc = accuracy(train_y_penguins, lda_penguin.predict(train_x_penguins))
penguins_test_acc = accuracy(test_y_penguins, lda_penguin.predict(test_x_penguins))
print(f"Testing LDA for Penguin dataset")
print(f"Train error for Penguin dataset and apriori probability distribution {prob_array_penguin} is {1 - penguins_train_acc:.4f}")
print(f"Test error for the same dataset:  {1 - penguins_test_acc:.4f}")



Testing LDA for iris dataset and apriori probability distribution
Train error for iris dataset and apriori probability distribution [0.33333333 0.33333333 0.33333333] is 0.0222
Test error for the same dataset:  0.0167
Testing LDA for Penguin dataset
Train error for Penguin dataset and apriori probability distribution [0.33333333 0.33333333 0.33333333] is 0.0195
Test error for the same dataset:  0.0219


### 4. Repetir la comparación con diferentes random seeds

In [ ]:
# Código para el punto 4

### 5. Comparar tiempos de predicción entre QDA y TensorizedQDA

In [ ]:
# Código para el punto 5

## Optimización matemática

### QDA

In [ ]:
# Código para la optimización de QDA

### LDA

In [ ]:
# Código para la optimización de LDA

## Preguntas teóricas

1. Demostración de la función a maximizar en LDA

2. Explicación de por qué QDA y LDA son "quadratic" y "linear"

3. Diferencias entre la implementación de QDA y la descripción teórica

## Ejercicio teórico

Cálculo de las derivadas de J respecto a cada parámetro de la red neuronal